# Deploy two models in the same online deployment
In this sample, we build an environment from scratch, write code for the server including a `score.py` file and a basic custom module, test the deployment locally, and deploy to the cloud. We'll see how the `score.py` file can be used as an entrypoint for multiple models and custom code. 

We'll use two simple binary image classification models called `cats` and `horses` that we create in the example [Train two image classification models in one pipeline](). 

# Environment 
To build the environment, we start with the Azure minimal Ubuntu 18.04 image. The inference server is pre-loaded and we will install the required additional libraries `libpng` via apt and packages `tensorflow`, `pillow`, and `keras-preprocessing` via pip in the Dockerfile.
## Inputs
### Dockerfile
```dockerfile
FROM "mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest"

USER root
RUN apt-get update
RUN apt-get install -y libpng-dev

USER dockeruser

COPY requirements.txt /tmp/requirements.txt

RUN pip install -r /tmp/requirements.txt
```
### requirements.txt
```
tensorflow-gpu>=2.8, <2.9
keras>= 2.8, <2.9
pillow>= 9.1, <9.2
keras-preprocessing>=1.1, <1.2
``` 
## Build
To build the environment, will call `az` with the environment file `environment.yaml` which contains the name of the environment `dualdeployment` as well as the directory "deploy" in which the Dockerfile and `requirements.txt` file is located. 

```yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: dualdeploy-env
build:
  path: ./deploy
```
Build the environment with the following command:

In [ ]:
!az ml environment create --file environment.yaml

# Configure code 
An online deployment requires a scoring file (usually called `score.py`), which should have an `init` method called once and a `run` method called for every request. Our `score.py` file follows this pattern. However, rather than handle the models entirely in `score.py` we import a minimal custom module `model.py`, which contains a handler for the `cats` and `horses` models. The `score.py` file can act as an entrypoint for extensive custom behavior by using a similar pattern.

## score.py (snippet)
```python
def init(): 
    global handle
    handle = Handle(os.getenv("AZUREML_MODEL_DIR"))

    logging.info("Init complete")

def run(raw_data): 
    logging.info("Request received")
    try:
        response = handle(raw_data)
        logging.info("Request processed")
        return response
    except ValueError as e:
        logging.info("Request failed")
        logging.info(str(e))
```

## model.py (snippet)

```python
class Handle(object):
    def __init__(self, azureml_model_dir):
        self.azureml_model_dir = azureml_model_dir
        self.models = {'cats' : Cats(azureml_model_dir), 
                       'horses' : Horses(azureml_model_dir)}
        self.valid_categories = set(self.models.keys())             

    def __call__(self, raw_data):
        raw_data = json.loads(raw_data)
        try:
            image = raw_data["image"]
            category = raw_data["category"]
        except KeyError:
            raise ValueError("Request must contain fields 'image' and 'category'")
        if category in self.valid_categories:
            return self.models[category].score(image).tolist()
        else:
            raise ValueError("No model for category")
```

# Deploy
Before we deploy to Azure, we'll first and test and deploy on our local machine.

## Create a local endpoint
The YAML file contains just "name" and "auth_mode." 
```yaml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: dualdeploy-endpt
version: 1
auth_mode: key
```
Deploy the local online endpoint with this command:

In [ ]:
!az ml online-endpoint create --file endpoint.yaml --local

## Create the deployment
The deployment YAML file integrates the endpoint, environment, code, and model:
```yaml 
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: dualdeploy-dplmt
endpoint_name: dualdeploy-endpt 
model:
  name: dualdeploy
  path: ./model/
code_configuration:
  code: ./deploy/code
  scoring_script: score.py
environment: azureml:dualdeploy-env:1
instance_type: Standard_F2s_v2
instance_count: 1
``` 

In [ ]:
!az ml online-deployment create --file deployment.yaml --local true

If the deployment fails, try checking the logs with this command:

In [ ]:
!az ml online-deployment get-logs --name cathorse --endpoint-name cathorse-endpoint  --local

## Test the deployment
In order to facilitate calling the endpoint with `invoke`, the endpoint can accept images formatted as `.json` data. The json must have the keys "category" and "image".

The directory `test-data` has directories for "cats", "horses", "not_cats", and "not_horses", each containing 5 png and json files selected from the test dataset. Let's try a few.

If your deployment fails upon invoking, check the logs with the command above.

### Cat Model

#### Cats

In [24]:
!az ml online-endpoint invoke --name dualdeploy-endpt --local true --request-file test_data/cats/0.json

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
"[[0.9998284578323364]]"


#### Not Cats

In [25]:
!az ml online-endpoint invoke --name dualdeploy-endpt --local true --request-file test_data/not_cats/0.json

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
"[[1.348734679496033e-09]]"


### Horse Model

#### Horses

In [26]:
!az ml online-endpoint invoke --name dualdeploy-endpt --local true --request-file test_data/horses/0.json

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
"[[0.999983549118042]]"


#### Not Horses

In [27]:
!az ml online-endpoint invoke --name dualdeploy-endpt --local true --request-file test_data/not_horses/0.json

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
"[[0.00010628611198626459]]"


# Deploy to Azure
Finally, we can push our deployment to Azure with no changes to the YAML by removing the `--local` flags.

## Delete the local endpoint

In [36]:
!az ml online-endpoint delete --name dualdeploy-endpt --local

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Are you sure you want to perform this operation? (y/n): ^C


## Create an online endpoint

In [29]:
!az ml online-endpoint create --file endpoint.yaml

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "auth_mode": "key",
  "id": "/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/v-alwallace-test/providers/Microsoft.MachineLearningServices/workspaces/valwallace/onlineEndpoints/dualdeploy-endpt",
  "identity": {
    "principal_id": "21dabe56-1ba8-459f-8564-8e231a732509",
    "tenant_id": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "type": "system_assigned"
  },
  "location": "eastus2",
  "name": "dualdeploy-endpt",
  "properties": {
    "AzureAsyncOperationUri": "https://management.azure.com/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:4de1aac5-5d2e-4819-8298-47038d76b5ae:36df4696-22f0-4a9d-a4d2-460580638b2d?api-version=2021-10-01",
    "azureml.onlineendpointid": "/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourcegroups/v-alwallace-tes

## Create an online deployment

In [30]:
!az ml online-deployment create --file deployment.yaml --all-traffic

Command group 'ml online-deployment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Check: endpoint dualdeploy-endpt exists
Uploading code (0.01 MBs): 100%|█████████| 7649/7649 [00:00<00:00, 40959.30it/s]


Uploading model (14.35 MBs): 100%|█| 14351714/14351714 [00:07<00:00, 1945970.14i


Creating/updating online deployment dualdeploy-dplmt ..........................................................Done (5m 25s)
{
  "app_insights_enabled": false,
  "code_configuration": {
    "code": "/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/v-alwallace-test/providers/Microsoft.MachineLearningServices/workspaces/valwallace/codes/a8370bb8-1301-4d53-841a-0bd425f5aad1/versions/1",
    "scoring_script": "score.py"
  },
  "endpoint_name": "dualdeploy-endpt",
  "environment": "azureml:/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/v-alwallace-test/providers/Microsoft.MachineLearningServices/workspaces/valwallace/env

## Test the deployment

In [35]:
!az ml online-endpoint invoke --name dualdeploy-endpt --request-file test_data/cats/1.json

Command group 'ml online-endpoint' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
"[[0.9987295866012573]]"
